```{contents}
```
## Self-Correction Loop 

A **self-correction loop** in LangGraph is a **cyclic execution pattern** where the system **generates an output, evaluates its own result, identifies errors or weaknesses, and iteratively improves the output** until a quality or termination condition is satisfied.

This pattern is essential for building **robust, autonomous, and production-grade LLM systems**.

---

### **1. Motivation**

LLMs are probabilistic and imperfect.
Single-pass generation often yields:

* Logical errors
* Missing information
* Hallucinations
* Poor structure

The self-correction loop introduces **feedback control**, allowing the system to **refine itself automatically**.

---

### **2. Conceptual Workflow**

```
Draft → Evaluate → Critique → Revise → (Repeat) → Final
```

This is implemented as a **cyclic LangGraph**.

---

### **3. State Design**

```python
class State(TypedDict):
    draft: str
    feedback: str
    iteration: int
    done: bool
```

The **state** stores both the evolving solution and its evaluation.

---

### **4. Graph Structure**

```
generate ──▶ critique ──▶ revise
   ▲                         │
   └──────────── loop ───────┘
                  │
                finish
```

---

### **5. Node Responsibilities**

| Node       | Responsibility          |
| ---------- | ----------------------- |
| `generate` | Produce initial draft   |
| `critique` | Identify flaws & gaps   |
| `revise`   | Improve using feedback  |
| `router`   | Decide continue vs stop |

---

### **6. Implementation Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    draft: str
    feedback: str
    iteration: int
    done: bool

def generate(state):
    return {"draft": f"Draft v{state['iteration']} answer"}

def critique(state):
    fb = "Needs improvement" if state["iteration"] < 2 else "Looks good"
    return {"feedback": fb}

def revise(state):
    return {
        "draft": state["draft"] + " [revised]",
        "iteration": state["iteration"] + 1,
        "done": state["iteration"] >= 2
    }

def route(state):
    return END if state["done"] else "generate"

builder = StateGraph(State)

builder.add_node("generate", generate)
builder.add_node("critique", critique)
builder.add_node("revise", revise)

builder.set_entry_point("generate")
builder.add_edge("generate", "critique")
builder.add_edge("critique", "revise")

builder.add_conditional_edges("revise", route, {
    "generate": "generate",
    END: END
})

graph = builder.compile()
result = graph.invoke({"draft": "", "feedback": "", "iteration": 0, "done": False})
```

---

### **7. Termination Strategies**

| Strategy          | Purpose                |
| ----------------- | ---------------------- |
| Max iterations    | Prevent infinite loops |
| Quality threshold | Stop when score high   |
| Human approval    | Manual termination     |
| Timeout           | Safety guard           |

```python
graph.invoke(input, config={"recursion_limit": 10})
```

---

### **8. Production Use Cases**

| Use Case          | Benefit                |
| ----------------- | ---------------------- |
| Code generation   | Eliminates bugs        |
| Report writing    | Improves clarity       |
| Planning          | Refines strategy       |
| Reasoning         | Reduces hallucination  |
| Autonomous agents | Continuous improvement |

---

### **9. Why It Works**

The loop introduces a **control system**:

> **Output → Measurement → Error → Correction**

This is identical to how **feedback controllers** stabilize physical systems.

---

### **10. Key Properties**

| Property   | Effect                 |
| ---------- | ---------------------- |
| Cyclic     | Enables refinement     |
| Stateful   | Preserves learning     |
| Autonomous | Reduces human effort   |
| Safe       | Controlled termination |

---

### **11. Relationship to Other Patterns**

| Pattern          | Difference             |
| ---------------- | ---------------------- |
| ReAct            | Action + observation   |
| Reflection loop  | High-level critique    |
| Planner-Executor | Structural improvement |
| Self-correction  | Quality refinement     |

---

### **12. Mental Model**

LangGraph self-correction behaves like:

> **An editor that never gets tired and never forgets past mistakes**



### Demonstration

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

# -----------------------------
# 1. Define State
# -----------------------------
class State(TypedDict):
    draft: str
    feedback: str
    iteration: int
    done: bool

# -----------------------------
# 2. Define Nodes
# -----------------------------
def generate(state: State):
    text = f"Answer version {state['iteration']}"
    return {"draft": text}

def critique(state: State):
    if state["iteration"] < 2:
        return {"feedback": "Improve clarity and completeness"}
    return {"feedback": "Satisfactory"}

def revise(state: State):
    improved = state["draft"] + " [improved]"
    new_iter = state["iteration"] + 1
    return {
        "draft": improved,
        "iteration": new_iter,
        "done": new_iter >= 3
    }

def router(state: State):
    return END if state["done"] else "generate"

# -----------------------------
# 3. Build Graph
# -----------------------------
builder = StateGraph(State)

builder.add_node("generate", generate)
builder.add_node("critique", critique)
builder.add_node("revise", revise)

builder.set_entry_point("generate")
builder.add_edge("generate", "critique")
builder.add_edge("critique", "revise")

builder.add_conditional_edges("revise", router, {
    "generate": "generate",
    END: END
})

graph = builder.compile()

# -----------------------------
# 4. Execute
# -----------------------------
result = graph.invoke({
    "draft": "",
    "feedback": "",
    "iteration": 0,
    "done": False
})

print(result)


{'draft': 'Answer version 2 [improved]', 'feedback': 'Satisfactory', 'iteration': 3, 'done': True}
